### Оптимизация портфеля генетическим алгоритмом

Здесь будет рассмотреть один из самых простых алгоритмов.

Рекомендую изучить и применять для своей работы `алгоритм роя частиц` (узнать о нем на [Habr](https://habr.com/ru/post/105639/))

Узнать больше о текущем походе:

1. https://www.researchgate.net/publication/286952225_A_heuristic_crossover_for_portfolio_selection
3. https://www.math.kth.se/matstat/seminarier/reports/M-exjobb12/121008.pdf

Мы продолжаем рассматривать Портфельную теорию от Марковица (Modern Portfolio Theory (MPT) by Harry Markowitz).

Сегодня мы погрузимся в задачи увеличения качества и скорости формирования портфеля. 

#### Разве это проблема?

Конечно, чем больше акций, тем больше вариантов их формирования.

Если мы будем минимизировать шаг в наборе (так, чтобы минимальный вклад в портфель бумагу был меньше 1%), то с увеличением количества акций, мы будем упираться в проблему "ожидания формирования", т.е. будем должго ждать).

Почему? Потому, что мы стараемся перебрать все доступные варианты. Это можно рассчитать по формуле - `количество оригинальных сочитаний из набора`: факториал от кол-ва элементов всего / (факториал от (кол-во элементов всего - кол-во элементов в ожидаемом списке) * кол-во элементов в ожидаемом списке)

N - кол-во элементов всего

K - кол-во элементов в ожидаемом списке

=> N! / (N-K)! * K1)



In [274]:
# пример

from math import factorial

# мы выбираем из 5 акций
stock = 5

# шаг выбор - 1% от акции (т.е. мы можем получить любой набор, где минимально будет 1% акции, а максимально 96%)
# например - 1%, 1%, 1%, 1%, 96%
step = 1

# шагов по одной акции
full_step = 100 / step
# шагов по всем вариантам
all_steps = int(full_step*stock)

# реализуем расчет количества комбинаций
# сколько комбинаций доступно, если у нас 100 вариантов на 1 акцию, а всего их 5
n = all_steps
k = stock

combs = int(factorial(n) / (factorial(n-k) * factorial(k)))

print('Из {} вариантов позиций для {} акций,при шаге в {}%, \nможно сформировать {} уникальных комбинаций'.format(all_steps, 
                                                                                             stock,
                                                                                             step,
                                                                                             combs))

Из 500 вариантов позиций для 5 акций,при шаге в 1%, 
можно сформировать 255244687600 уникальных комбинаций


### Методика

1. Делаем расчет на 3, 6, 12, 24 и 36 месяцев по каждой акции
3. Определяем **Ген**: % одной акции в списке из 5 акций
4. Определяем **Хромосому**: список % из 5 акций. В сумме равно 1 (или 100%)
5. **Стартовая популяция**: случайный набор хромосом
6. **Функция обучения**
**Коэф. Шарпа** - S - рассчитывается для **Функции обучения** из результатов `максимально возможного возврата и минимально возможного риска` (среднее по этим результатам за все время обучения:
                
S = (µ − r)/σ
    
µ - средний возможный возврат по портфелю 
r - безрисковое значение µ за этот же период
σ - стандартное отклонение по портфелю


7. Выбираем **Идеальную популярность**: выбираем по результату функции обучения. Лучшая та, которая больше возвращает с минимальным риском
    
8. **Мутации**: Перемешивание лучших генов от 0 до 5 вариантов

9. **Пересечения** используется для изменение направления поиска и перемешивания результатов в генах. 


A = Лучший ген  + β ∗ ( Лучший ген − Худший ген)

B = Худший ген - β ∗ ( Лучший ген − Худший ген)

β случайное число от 0 до 1.

10. **Следующее покаление**: результат мутации генов.
    
11. Процесс повторяется до нахождения лучшей комбинации


#### Набор данных

In [275]:
import numpy as np
import pandas as pd
from functools import reduce

In [64]:
# все же лучше брать из yahoo
import yfinance as yf

In [276]:
yf.Ticker('VSCO')

yfinance.Ticker object <VSCO>

In [277]:
yf.Ticker('VSCO').history(period=time_frame)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-08-03,48.000000,63.740002,47.970001,58.230000,25163400,0,0
2021-08-04,61.439999,64.529999,57.500000,57.919998,5179300,0,0
2021-08-05,59.900002,62.939999,58.750000,58.770000,3749600,0,0
2021-08-06,59.000000,59.419998,56.500000,57.669998,4238800,0,0
2021-08-09,57.619999,75.430000,56.349998,69.239998,7722800,0,0
...,...,...,...,...,...,...,...
2021-12-13,50.500000,50.770000,49.400002,49.900002,1581000,0,0
2021-12-14,49.380001,50.540001,48.830002,49.980000,1017800,0,0
2021-12-15,50.220001,50.730000,47.380001,49.070000,1728300,0,0


In [17]:
yf.Ticker('VSCO').history(period=time_frame)['Close'].pct_change()

Date
2021-08-03         NaN
2021-08-04   -0.005324
2021-08-05    0.014675
2021-08-06   -0.018717
2021-08-09    0.200624
                ...   
2021-12-10   -0.023162
2021-12-13   -0.014029
2021-12-14    0.001603
2021-12-15   -0.018207
2021-12-16   -0.000204
Name: Close, Length: 96, dtype: float64

In [278]:
stock_analysis = ['BABA', 'RHM.DE', 'NVDA', 'AFLT.ME', 'MOEX.ME']
n_stock = len(stock_analysis)
basis = np.eye(len(stock_analysis))
time_frame = '3y'

In [279]:
tickers = {stock: yf.Ticker(stock) for stock in stock_analysis}

In [363]:
data = pd.DataFrame({stock: tickers[stock].history(period=time_frame)['Close'].pct_change() for stock in stock_analysis})
data.head()

,BABA,RHM.DE,NVDA,AFLT.ME,MOEX.ME
Date,,,,,
2018-12-17,NaN,NaN,NaN,NaN,NaN
2018-12-18,-0.021947,0.012984,0.023402,0.005542,-0.002020
2018-12-19,-0.026133,-0.005896,-0.057370,0.003937,0.005952
2018-12-20,-0.014802,-0.011088,-0.024619,0.000196,-0.012071
2018-12-21,-0.023018,0.010170,-0.040933,0.001764,-0.002156


###  Делаем расчет на 3, 6, 12, 24 и 36 месяцев по каждой акции



In [330]:
def hist_return(months, data):
    ''' 
        Input: Номера месяцев.
               Pandas Data Frame
        Output: Расчет за переданные месяцы
    '''
    idx=[]
    df=pd.DataFrame()
    
    # агрегат по месяцу
    tmp = data.groupby([pd.Grouper(freq='M')]).sum()
    

    for mon in months:
        
        # какая формула была бы правильная?
        
        # temp=(tmp.iloc[-mon,:] - tmp.iloc[-1,:] )/(tmp.iloc[-mon,:])
        
        temp=(tmp.iloc[0,:] - tmp.iloc[mon,:])/(tmp.iloc[mon,:])
        
        
        idx.append(str(mon)+'_mon_return')
        df=pd.concat([df, temp.to_frame().T], ignore_index=True)
    df.index=idx
    
    return df    

In [364]:
hist_stock_returns= data.dropna()#hist_return([3,6, 12, 24, 36], data).replace([-np.inf, np.inf], 0)
hist_stock_returns

,BABA,RHM.DE,NVDA,AFLT.ME,MOEX.ME
Date,,,,,
2018-12-18,-0.021947,0.012984,0.023402,0.005542,-0.002020
2018-12-19,-0.026133,-0.005896,-0.057370,0.003937,0.005952
2018-12-20,-0.014802,-0.011088,-0.024619,0.000196,-0.012071
2018-12-21,-0.023018,0.010170,-0.040933,0.001764,-0.002156
2018-12-27,0.003261,-0.002065,-0.014500,-0.005148,-0.003198
...,...,...,...,...,...
2021-12-13,-0.022549,-0.013461,-0.067455,-0.017673,-0.016430
2021-12-14,0.035504,0.007938,0.006250,0.002037,-0.016972
2021-12-15,-0.032470,-0.007138,0.074884,-0.002710,-0.012235


### Определяем **Ген**

In [365]:
gene = np.random.rand()
gene

0.5182219914552796

In [366]:
import time

def gen_mc_grid(rows, cols, n, N):
    """
    Методом Монте-Карло создаем сетку с параметрами
    """
    np.random.seed(seed=28)  # seed
    
    # создаем строки и колонки    
    layouts = np.zeros((n, rows * cols), dtype=np.int32)  
    
    # стартовая позиция по X/Y
    positionX = np.random.randint(0, cols, size=(N * n * 2))
    positionY = np.random.randint(0, rows, size=(N * n * 2))
    ind_rows = 0  
    ind_pos = 0  
    
    while ind_rows < n:
        layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
        if np.sum(layouts[ind_rows, :]) == N:
            ind_rows += 1
        ind_pos += 1
        if ind_pos >= N * n * 2:
            print("Not enough positions")
            break
    return layouts


def gen_mc_grid_with_NA_loc(rows, cols, n, N,NA_loc):
    """
    Методом Монте-Карло создаем сетку с параметрами
    """
    
    np.random.seed(seed=28)  # seed
    
    # повторение, но уже с NA
    layouts = np.zeros((n, rows * cols), dtype=np.int32) 
    layouts_NA= np.zeros((n, rows * cols), dtype=np.int32)
    
    for i in NA_loc:
        layouts_NA[:,i-1]=2

    # стартовая позиция по X/Y
    positionX = np.random.randint(0, cols, size=(N * n * 2))
    positionY = np.random.randint(0, rows, size=(N * n * 2))
    ind_rows = 0 
    ind_pos = 0  
    N_count=0
    
    while ind_rows < n:
        cur_state=layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]
        if cur_state!=1 and cur_state!=2:
            layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]=1
            layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
            N_count+=1
            if np.sum(layouts[ind_rows, :]) == N:
                ind_rows += 1
                N_count=0
        ind_pos += 1
        if ind_pos >= N * n * 2:
            print("Not enough positions")
            break
    return layouts,layouts_NA

In [367]:
gen_mc_grid(5, 5, 100, 50)
gen_mc_grid_with_NA_loc(5, 5, 100, 50,range(10))

Not enough positions
Not enough positions


(array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[2, 2, 2, ..., 1, 1, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        ...,
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2]]))

### Определяем **Хромосому**:

In [368]:
def chromosome(n):
    ''' Рандомный набор из позиций, но в сумме 1'''
    ch = np.random.rand(n)
    return ch/sum(ch)

In [369]:
child=chromosome(5)
print(child,sum(child))

[0.19073989 0.3446973  0.31615295 0.13751315 0.0108967 ] 0.9999999999999999


### **Стартовая популяция**: 


In [370]:
# кол-во акций
n=5

# базовая популяция (кол-во шагов)
pop_size=100 

population = np.array([chromosome(n) for _ in range(pop_size)])
print(population.shape)
print(population)

(100, 5)
[[3.79144094e-01 2.92334623e-02 2.14760167e-02 3.77286657e-01
  1.92859770e-01]
 [2.14026325e-01 1.77679162e-01 2.71598987e-01 3.04160421e-01
  3.25351056e-02]
 [2.32179992e-01 2.96786556e-01 3.03116422e-01 8.69199458e-02
  8.09970832e-02]
 [8.83780243e-02 3.35638154e-01 3.10406720e-01 1.42191582e-01
  1.23385520e-01]
 [1.77268184e-01 3.34878692e-01 1.11872379e-01 1.26058188e-01
  2.49922557e-01]
 [2.61259093e-01 1.55920172e-01 2.73688611e-01 2.78815337e-01
  3.03167864e-02]
 [3.09053194e-01 2.50878542e-01 2.22240316e-01 1.94821881e-01
  2.30060675e-02]
 [4.08981006e-02 2.14668271e-01 2.86389480e-01 1.84929829e-01
  2.73114320e-01]
 [6.93125007e-02 2.61070014e-01 2.48803367e-01 2.69120634e-01
  1.51693484e-01]
 [2.09866308e-01 2.24299843e-01 1.95586718e-01 1.59521503e-01
  2.10725628e-01]
 [1.31019996e-01 4.66864531e-01 1.53678912e-01 4.84918131e-04
  2.47951643e-01]
 [2.56278583e-01 2.06204079e-01 1.82596358e-01 8.89242943e-02
  2.65996686e-01]
 [6.86175533e-02 2.36663285e-01

### **Функция обучения**

 #### Расчет параметров проверки
 
 - mean
 - std
 - cov

In [372]:
# проверка на подходяшие типы для расчетов
print(hist_stock_returns.info())
cols=hist_stock_returns.columns
hist_stock_returns[cols] = hist_stock_returns[cols].apply(pd.to_numeric, errors='coerce')
print(hist_stock_returns.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 720 entries, 2018-12-18 to 2021-12-17
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BABA     720 non-null    float64
 1   RHM.DE   720 non-null    float64
 2   NVDA     720 non-null    float64
 3   AFLT.ME  720 non-null    float64
 4   MOEX.ME  720 non-null    float64
dtypes: float64(5)
memory usage: 33.8 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 720 entries, 2018-12-18 to 2021-12-17
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BABA     720 non-null    float64
 1   RHM.DE   720 non-null    float64
 2   NVDA     720 non-null    float64
 3   AFLT.ME  720 non-null    float64
 4   MOEX.ME  720 non-null    float64
dtypes: float64(5)
memory usage: 33.8 KB
None


#### Cov

In [373]:
cov_hist_return=hist_stock_returns.cov()
cov_hist_return

,BABA,RHM.DE,NVDA,AFLT.ME,MOEX.ME
BABA,0.000568,0.000104,0.000299,0.000043,0.000025
RHM.DE,0.000104,0.000509,0.000106,0.000150,0.000057
NVDA,0.000299,0.000106,0.000905,0.000039,0.000026
AFLT.ME,0.000043,0.000150,0.000039,0.000455,0.000086
MOEX.ME,0.000025,0.000057,0.000026,0.000086,0.000224


#### mean

In [374]:
mean_hist_return=hist_stock_returns.mean()
mean_hist_return

BABA       0.000203
RHM.DE     0.000337
NVDA       0.003009
AFLT.ME   -0.000565
MOEX.ME    0.001035
dtype: float64

#### std

In [375]:
sd_hist_return=hist_stock_returns.std()
sd_hist_return

BABA       0.023830
RHM.DE     0.022565
NVDA       0.030079
AFLT.ME    0.021339
MOEX.ME    0.014951
dtype: float64

#### Ожидаемая доходность

In [376]:
def mean_portfolio_return(child, mean_hist_return):
    return np.sum(np.multiply(child,mean_hist_return))

In [377]:
mean_portfolio_return(population[0], mean_hist_return)

0.00013783216132251235

#### Вариация по портфелю / риск

In [378]:
def var_portfolio_return(child, sd_hist_return, cov_hist_return):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(5):
        for j in range(5):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

In [379]:
var_portfolio_return(population[0], sd_hist_return, cov_hist_return)

0.0003518991267048853

#### Риск на который мы согласны

In [380]:
rf = 0.05

#### функция обучения

In [381]:
def fitness_fuction(child, mean_hist_return, sd_hist_return, cov_hist_return):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child, mean_hist_return)-rf)/np.sqrt(var_portfolio_return(child, sd_hist_return, cov_hist_return))

In [382]:
fitness_fuction(population[7], mean_hist_return, sd_hist_return, cov_hist_return)

-2.7315615558997983

### Выбираем **Идеальную популярность**: 

In [383]:
def Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return, frac=0.3):
    ''' 
        Input: популяция (набор % по каждой акции)
        Output: Лучшая популяция
    
    '''
    population = sorted(population,key = lambda x: fitness_fuction(x, mean_hist_return, sd_hist_return, cov_hist_return),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

In [384]:
print(len(Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return, frac=0.3)))
Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return,  frac=0.3)

30


[array([0.10664749, 0.07839368, 0.51302508, 0.23802569, 0.06390806]),
 array([0.28015513, 0.01671726, 0.44411077, 0.04353953, 0.21547731]),
 array([0.05699583, 0.11824051, 0.49849766, 0.08234964, 0.24391636]),
 array([0.23826409, 0.01378074, 0.44463741, 0.16390969, 0.13940808]),
 array([0.44103368, 0.10583632, 0.31901015, 0.02788393, 0.10623592]),
 array([0.23589584, 0.01175771, 0.03796588, 0.69431311, 0.02006746]),
 array([0.12018326, 0.51144007, 0.27167912, 0.09275341, 0.00394414]),
 array([0.14690183, 0.00994784, 0.38855501, 0.36925705, 0.08533828]),
 array([0.15317709, 0.44834217, 0.29670288, 0.02178309, 0.07999477]),
 array([0.12293407, 0.01473315, 0.43050696, 0.17023784, 0.26158798]),
 array([0.51763653, 0.32286116, 0.01915463, 0.04637317, 0.09397451]),
 array([0.23217108, 0.5138503 , 0.14771418, 0.09391621, 0.01234823]),
 array([0.14310097, 0.16925152, 0.39876889, 0.1258069 , 0.16307173]),
 array([0.02371265, 0.47083386, 0.15266216, 0.33056078, 0.02223054]),
 array([0.03956081, 

In [385]:
[fitness_fuction(x, mean_hist_return, sd_hist_return, cov_hist_return) for x in population][:3]

[-2.6580433630690434, -2.5027156507234745, -2.385507961556622]

### **Мутации**:


In [386]:
def mutation(parent):
    ''' 
        Input: Родительский набор
        Output: Мутированный набор
    '''
    child=parent.copy()
    n=np.random.choice(range(5),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(5),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

In [387]:
mutation(population[1]),population[1]

(array([0.27159899, 0.17767916, 0.21402632, 0.30416042, 0.03253511]),
 array([0.21402632, 0.17767916, 0.27159899, 0.30416042, 0.03253511]))

### **Пересечения** 

In [388]:
def Heuristic_crossover(parent1,parent2, 
                        mean_hist_return=mean_hist_return,
                        sd_hist_return=sd_hist_return,
                        cov_hist_return=cov_hist_return):
    ''' A = Лучший ген  + β ∗ ( Лучший ген − Худший ген)
        B = Худший ген - β ∗ ( Лучший ген − Худший ген)

        β случайное число от 0 до 1.

        Input: 2 родителя
        Output: 2 ребенка
    '''
    ff1=fitness_fuction(parent1, mean_hist_return, sd_hist_return, cov_hist_return)
    ff2=fitness_fuction(parent2, mean_hist_return, sd_hist_return, cov_hist_return)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

In [389]:
for i in population[:30]:
    for j in population[:30]:
        print(Heuristic_crossover(i,j, mean_hist_return, sd_hist_return, cov_hist_return))

(array([0.37914409, 0.02923346, 0.02147602, 0.37728666, 0.19285977]), array([0.37914409, 0.02923346, 0.02147602, 0.37728666, 0.19285977]))
(array([0.19390382, 0.19576988, 0.30208086, 0.29524871, 0.01299673]), array([ 0.39926659,  0.01114275, -0.00900586,  0.38619837,  0.21239815]))
(array([0.23003067, 0.30069947, 0.30723536, 0.08267339, 0.07936111]), array([0.38129342, 0.02532055, 0.01735708, 0.38153321, 0.19449574]))
(array([-0.17707696,  0.61537043,  0.57418611, -0.07243855,  0.05995898]), array([ 0.64459908, -0.25049881, -0.24230337,  0.59191679,  0.25628631]))
(array([ 0.46144851, -0.0953775 , -0.0153784 ,  0.47971201,  0.16959538]), array([0.09496377, 0.45948965, 0.1487268 , 0.02363284, 0.27318694]))
(array([ 0.19460114,  0.22755504,  0.41630196,  0.22313483, -0.06159298]), array([ 0.44580205, -0.04240141, -0.12113733,  0.43296716,  0.28476953]))
(array([0.30129176, 0.27542214, 0.2444717 , 0.17461687, 0.00419753]), array([ 0.38690552,  0.00468987, -0.00075537,  0.39749167,  0.2116

(array([0.25667613, 0.32265647, 0.3178652 , 0.03029503, 0.07250716]), array([0.03989495, 0.09371787, 0.18734428, 0.53140328, 0.14763963]))
(array([-0.1257411 ,  0.5612735 ,  0.52317429, -0.03093161,  0.07222492]), array([ 0.59326322, -0.19640189, -0.19129155,  0.55040985,  0.24402037]))
(array([0.01961184, 0.42208768, 0.33164584, 0.0535475 , 0.17310713]), array([ 0.2827925 ,  0.09122963,  0.25035986,  0.3928045 , -0.01718651]))
(array([0.31671567, 0.27394719, 0.29883074, 0.05442787, 0.05607854]), array([0.00384235, 0.35847752, 0.31469241, 0.17468366, 0.14830406]))
(array([0.08837802, 0.33563815, 0.31040672, 0.14219158, 0.12338552]), array([0.08837802, 0.33563815, 0.31040672, 0.14219158, 0.12338552]))
(array([0.02200395, 0.33620524, 0.45865185, 0.15423835, 0.02890061]), array([ 0.24364226,  0.3343116 , -0.03637275,  0.11401142,  0.34440747]))
(array([0.04006787, 0.38585883, 0.32066729, 0.10401323, 0.14939279]), array([0.30956925, 0.1056995 , 0.26342805, 0.31699369, 0.00430952]))
(array(

(array([ 0.37279716,  0.19272557,  0.22442206,  0.28924983, -0.07919461]), array([ 0.1587487 ,  0.3879998 ,  0.2170959 , -0.02783362,  0.26398922]))
(array([ 0.32633318,  0.24879322,  0.25168632,  0.19312214, -0.01993486]), array([0.20998048, 0.26283451, 0.05341518, 0.20456712, 0.26920272]))
(array([ 0.3375602 ,  0.32216195,  0.20316621,  0.20122252, -0.06411088]), array([0.23951133, 0.07698509, 0.26877095, 0.17920773, 0.2355249 ]))
(array([ 0.01496222,  0.63606564,  0.21308358, -0.0837089 ,  0.21959746]), array([ 0.43940482,  0.08014985,  0.22629891,  0.31827668, -0.06413026]))
(array([ 0.35852777,  0.31728521,  0.16449708,  0.21623499, -0.05654505]), array([0.19842403, 0.10238783, 0.35135888, 0.14694042, 0.20088883]))
(array([ 0.42393573,  0.28651429,  0.20189288,  0.22053817, -0.13288106]), array([0.01872278, 0.16082011, 0.27366222, 0.12983185, 0.41696305]))
(array([0.43809579, 0.17105597, 0.02101636, 0.34856261, 0.02126927]), array([0.30678865, 0.25227934, 0.22577156, 0.19212391, 0

(array([0.24981464, 0.21283759, 0.21924551, 0.20144809, 0.11665417]), array([ 0.05715888,  0.26811571,  0.10514806, -0.00074806,  0.5703254 ]))
(array([0.00537525, 0.0771086 , 0.20473253, 0.60267055, 0.11011306]), array([0.26888214, 0.26677903, 0.19294725, 0.03162932, 0.23976227]))
(array([-0.11156363,  0.89472355,  0.28292979, -0.36790305,  0.30181335]), array([ 0.62172772, -0.39862556, -0.10777486,  0.74567463,  0.13899807]))
(array([ 0.28649188, -0.07478326,  0.37454468,  0.56927292, -0.15552621]), array([ 0.05855444,  0.71932696,  0.05073322, -0.26462758,  0.43601296]))
(array([0.24534407, 0.27465409, 0.32256291, 0.09816785, 0.05927108]), array([ 0.11785592,  0.48899699,  0.13423242, -0.01076298,  0.26967765]))
(array([0.05189099, 0.22335299, 0.44451244, 0.26344431, 0.01679927]), array([ 0.16750703,  0.57914969,  0.01957319, -0.12076781,  0.35453789]))
(array([ 0.1099982 ,  0.52685779,  0.17268179, -0.05659356,  0.24705578]), array([0.19828998, 0.27488543, 0.0928695 , 0.18313666, 0

(array([0.06083177, 0.02099287, 0.18559778, 0.7061517 , 0.02642588]), array([ 0.07217687,  0.3352582 ,  0.23817549, -0.03133682,  0.38572627]))
(array([ 0.47156897, -0.10261815, -0.07570129,  0.55668424,  0.15006622]), array([ 0.14049002,  0.36969291,  0.27240153, -0.0859432 ,  0.30335874]))
(array([ 0.20811424,  0.15884855,  0.30176407,  0.37011094, -0.0388378 ]), array([0.23882698, 0.25667191, 0.14505914, 0.02750387, 0.3319381 ]))
(array([ 0.2316543 ,  0.33895135,  0.39460042,  0.08224572, -0.04745179]), array([0.2334406 , 0.19567651, 0.08374022, 0.0981286 , 0.38901407]))
(array([0.01877609, 0.38273237, 0.37550404, 0.16566105, 0.05732645]), array([0.30251684, 0.19074708, 0.1101269 , 0.06998492, 0.32662426]))
(array([0.23909635, 0.22706201, 0.18226159, 0.08983263, 0.26174742]), array([0.17108673, 0.34565798, 0.10483501, 0.12967995, 0.24874033]))
(array([ 0.27857295,  0.10587921,  0.33383491,  0.39204182, -0.11032889]), array([ 0.21560104,  0.28788226,  0.11507792, -0.0197721 ,  0.4012

(array([ 0.28250232,  0.15691156,  0.27272724, -0.01404631,  0.30190519]), array([ 0.20103673,  0.31004171, -0.0072697 ,  0.30583798,  0.19035329]))
(array([-0.03199419,  0.22138795,  0.3097196 ,  0.19824117,  0.30264548]), array([ 0.32787221,  0.27602452, -0.00517821,  0.20466273,  0.19661875]))
(array([0.23666534, 0.22264712, 0.23648612, 0.02088371, 0.28331771]), array([0.22351002, 0.27594336, 0.02159929, 0.27543805, 0.20350928]))
(array([ 0.35799692,  0.17798086,  0.24771155,  0.23430632, -0.01799566]), array([0.19876407, 0.27879004, 0.04692905, 0.19601469, 0.27950214]))
(array([ 0.21828081,  0.39088746,  0.33978747, -0.05378882,  0.10483308]), array([ 0.23147232,  0.19970856, -0.03764865,  0.32325053,  0.28321724]))
(array([0.22726046, 0.26074918, 0.08286118, 0.20286738, 0.22626179]), array([0.22726046, 0.26074918, 0.08286118, 0.20286738, 0.22626179]))
(array([0.26843505, 0.14711848, 0.25140259, 0.18543192, 0.14761196]), array([0.22684375, 0.2618992 , 0.08115544, 0.20304384, 0.2270

(array([0.12799126, 0.19600973, 0.25655116, 0.15859712, 0.26085072]), array([0.26189263, 0.2066502 , 0.17935999, 0.08587531, 0.26622188]))
(array([0.17528392, 0.17066961, 0.273603  , 0.12701639, 0.25342708]), array([0.02693894, 0.26244953, 0.201392  , 0.22856826, 0.28065128]))
(array([0.07566306, 0.17230949, 0.29887678, 0.19177677, 0.2613739 ]), array([0.29085715, 0.26198766, 0.12966223, 0.05722575, 0.26026721]))
(array([ 0.51597343,  0.19560847,  0.17224185,  0.29590057, -0.17972432]), array([-0.05286758,  0.1968691 ,  0.29285236,  0.0871012 ,  0.47604492]))
(array([0.26961813, 0.40056682, 0.20123214, 0.01943363, 0.10914928]), array([0.08647984, 0.12573587, 0.27136029, 0.20270883, 0.31371518]))
(array([0.08490515, 0.16302368, 0.34194972, 0.13094238, 0.27917907]), array([ 0.27596062,  0.29418136, -0.00577375,  0.22747313,  0.20815864]))
(array([0.36907608, 0.11203908, 0.24697671, 0.20820902, 0.06369911]), array([0.03254756, 0.23268527, 0.25603492, 0.13294749, 0.34578476]))
(array([ 0.1

(array([0.26280727, 0.27535658, 0.25481301, 0.04086626, 0.16615688]), array([0.22427686, 0.22700045, 0.15222533, 0.10865085, 0.2878465 ]))
(array([ 0.3587052 ,  0.16946776,  0.20401231,  0.29389896, -0.02608423]), array([ 0.22496456,  0.2910697 ,  0.23958124, -0.0103826 ,  0.25476711]))
(array([0.22035433, 0.33425702, 0.21843136, 0.06730526, 0.15965202]), array([0.25630756, 0.26010555, 0.2326604 , 0.05535179, 0.1955747 ]))
(array([ 0.27147543,  0.26693817,  0.32761227, -0.03835376,  0.17232789]), array([ 0.20995426,  0.25832675, -0.01293696,  0.29728387,  0.24737209]))
(array([ 0.24720647,  0.32296003,  0.22282343, -0.00906746,  0.21607752]), array([0.27498109, 0.08982421, 0.25868753, 0.25073856, 0.12576861]))
(array([0.05987339, 0.62296129, 0.20562618, 0.02693883, 0.0846003 ]), array([0.33960968, 0.10689139, 0.24333012, 0.06886979, 0.24129901]))
(array([0.24182603, 0.07609658, 0.3534652 , 0.27709769, 0.0515145 ]), array([ 0.26024179,  0.35721366,  0.17196457, -0.05268143,  0.2632614 ]

(array([ 0.75293586,  0.47607707, -0.2144528 , -0.16695874,  0.15239861]), array([ 0.02595976,  0.00270426,  0.50729604,  0.49214724, -0.02810731]))
(array([ 0.72260474,  0.39359619, -0.18041121, -0.09950266,  0.16371294]), array([ 0.10408498,  0.18014351,  0.42180616,  0.34069771, -0.04673237]))
(array([ 0.76409469,  0.37879334, -0.11899703, -0.02525609,  0.00136509]), array([-0.20556006,  0.15873609,  0.42454115,  0.25655909,  0.36572374]))
(array([ 0.87832786,  0.37257415, -0.16560528, -0.13283445,  0.04753771]), array([-0.29137883,  0.21135702,  0.43356328,  0.44832825,  0.19813028]))
(array([ 0.62880957,  0.35846357, -0.04457632,  0.00550162,  0.05180157]), array([0.09869327, 0.18869743, 0.25931768, 0.20039305, 0.25289857]))
(array([ 0.76282295,  0.23153637, -0.06615865,  0.07547481, -0.00367548]), array([-0.11416642,  0.55818931,  0.23899219, -0.02861672,  0.34560164]))
(array([0.54491806, 0.33503827, 0.00209397, 0.04193152, 0.07601819]), array([0.22899705, 0.19402697, 0.19965702

(array([ 0.26430049,  0.19719659,  0.29495145,  0.24683653, -0.00328506]), array([0.01411692, 0.10007036, 0.17874059, 0.53210226, 0.17496987]))
(array([ 0.29789216,  0.36618396,  0.3426808 , -0.06497935,  0.05822243]), array([-0.00132109,  0.05019038,  0.16252868,  0.62667766,  0.16192436]))
(array([ 0.10743492,  0.50728358,  0.39645862, -0.12203847,  0.11086135]), array([ 0.04533419, -0.05205764,  0.11604116,  0.73900842,  0.15167387]))
(array([-0.01100368, -0.02421291,  0.26235476,  0.70770137,  0.06516045]), array([ 0.25266294,  0.47867939,  0.05161067, -0.10686481,  0.32391181]))
(array([ 0.36129533,  0.17438206,  0.31006907,  0.17923896, -0.02498542]), array([-0.03564515,  0.1011259 ,  0.1657126 ,  0.57435474,  0.19445191]))
(array([ 0.47811105,  0.34159849,  0.23616177,  0.00137614, -0.05724745]), array([-0.10466677,  0.02886784,  0.18817161,  0.6682241 ,  0.21940322]))
(array([0.08594361, 0.03236067, 0.12475928, 0.74068625, 0.01625018]), array([ 0.01934557,  0.30189538,  0.36372

In [390]:
def Arithmetic_crossover(parent1,parent2):
    ''' A = Лучший ген  + β ∗ ( Лучший ген − Худший ген)
        B = Худший ген - β ∗ ( Лучший ген − Худший ген)

        β случайное число от 0 до 1.

        Input: 2 родителя
        Output: 2 ребенка
    '''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

In [391]:
Arithmetic_crossover(population[2],population[3])

(array([0.1340034 , 0.32331135, 0.30809366, 0.12465504, 0.10993655]),
 array([0.18655461, 0.30911336, 0.30542948, 0.10445649, 0.09444605]))

### **Следующее покаление**

In [392]:
def next_generation(pop_size,
                    elite,
                    crossover=Arithmetic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array)
    '''
    
    new_population=[]
    elite_range=range(len(elite))
    
    # цикл создания покаления через пересечение генов
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: 
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(5):
                
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if chk>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            new_population.extend([c1,c2])
    return new_population

In [393]:
elite=Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return)
next_generation(100,elite)[:3]

[array([0.22224032, 0.25087854, 0.30905319, 0.19482188, 0.02300607]),
 array([0.10664749, 0.23802569, 0.51302508, 0.07839368, 0.06390806]),
 array([0.10623592, 0.10583632, 0.31901015, 0.02788393, 0.44103368])]

In [394]:
elite=Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return)
next_generation(100,elite, Heuristic_crossover)[:3]

[array([0.43012177, 0.38463287, 0.03239226, 0.04445697, 0.10839614]),
 array([0.43012177, 0.38463287, 0.03239226, 0.04445697, 0.10839614]),
 array([0.44436541, 0.02071148, 0.40652308, 0.02282369, 0.10557635])]

### Процесс повторяется до нахождения лучшей комбинации

подход Heuristic_crossover

In [395]:
# кол-во акций
n=5

# базовая популяция
pop_size=100
population = np.array([chromosome(n) for _ in range(pop_size)])

# базовые лучшие поколения
elite = Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 and Expected_risk > 0.0005) or iteration <= 40:
    print('Итерация:',iteration)
    population = next_generation(100,elite, Heuristic_crossover)
    elite = Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return)
    Expected_returns=mean_portfolio_return(elite[0], mean_hist_return)
    Expected_risk=var_portfolio_return(elite[0], sd_hist_return, cov_hist_return)
    print('Ожидаемый доход {} с риском {}\n'.format(round(Expected_returns, 3), round(Expected_risk,5)))
    iteration+=1


print('Сформированный портфель:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(5))]

Итерация: 0
Ожидаемый доход -0.001 с риском 0.00128

Итерация: 1
Ожидаемый доход 0.004 с риском 0.00455

Итерация: 2
Ожидаемый доход 0.01 с риском 0.0325

Итерация: 3
Ожидаемый доход 0.019 с риском 0.16391

Итерация: 4
Ожидаемый доход 0.045 с риском 0.86462

Итерация: 5
Ожидаемый доход 0.074 с риском 1.50891

Итерация: 6
Ожидаемый доход 0.092 с риском 3.40092

Итерация: 7
Ожидаемый доход 0.123 с риском 6.12344

Итерация: 8
Ожидаемый доход 0.16 с риском 11.19999

Итерация: 9
Ожидаемый доход 0.218 с риском 11.17313

Итерация: 10
Ожидаемый доход 0.218 с риском 11.17313

Итерация: 11
Ожидаемый доход 0.507 с риском 58.58786

Итерация: 12
Ожидаемый доход 0.613 с риском 79.0357

Итерация: 13
Ожидаемый доход 0.801 с риском 130.42025

Итерация: 14
Ожидаемый доход 1.069 с риском 220.13578

Итерация: 15
Ожидаемый доход 1.491 с риском 431.41707

Итерация: 16
Ожидаемый доход 1.747 с риском 570.99319

Итерация: 17
Ожидаемый доход 2.129 с риском 849.25454

Итерация: 18
Ожидаемый доход 2.976 с риском 

[None, None, None, None, None]

### Итоговый портфель

In [397]:
print('Веса(%) в рекомендуемом портфеле:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(5))]

print('\nОжидаемый доход {} с риском {}\n'.format(Expected_returns,Expected_risk))

Веса(%) в рекомендуемом портфеле:

BABA : -2201.172300960458
RHM.DE : -5.845510821530581
NVDA : 4188.43849648327
AFLT.ME : -4661.072250980149
MOEX.ME : 2680.6515662788843

Ожидаемый доход 17.56164943533257 с риском 52246.99213015754



подход Arithmetic_crossover

In [328]:
# кол-во акций
n=5

# базовая популяция
pop_size=100
population = np.array([chromosome(n) for _ in range(pop_size)])

# базовые лучшие поколения
elite = Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 and Expected_risk > 0.0005) or iteration <= 40:
    print('Итерация:',iteration)
    population = next_generation(100,elite)
    elite = Select_elite_population(population, mean_hist_return, sd_hist_return, cov_hist_return)
    Expected_returns=mean_portfolio_return(elite[0], mean_hist_return)
    Expected_risk=var_portfolio_return(elite[0], sd_hist_return, cov_hist_return)
    print('Ожидаемый доход {} с риском {}\n'.format(round(Expected_returns, 3), round(Expected_risk,5)))
    iteration+=1


print('Сформированный портфель:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(5))]

Итерация: 0
Ожидаемый доход 0.475 с риском 0.00588

Итерация: 1
Ожидаемый доход 0.475 с риском 0.00588

Итерация: 2
Ожидаемый доход 0.476 с риском 0.00596

Итерация: 3
Ожидаемый доход 0.471 с риском 0.00619

Итерация: 4
Ожидаемый доход 0.467 с риском 0.00613

Итерация: 5
Ожидаемый доход 0.465 с риском 0.00611

Итерация: 6
Ожидаемый доход 0.464 с риском 0.00613

Итерация: 7
Ожидаемый доход 0.465 с риском 0.00614

Итерация: 8
Ожидаемый доход 0.464 с риском 0.00614

Итерация: 9
Ожидаемый доход 0.464 с риском 0.00614

Итерация: 10
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 11
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 12
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 13
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 14
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 15
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 16
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 17
Ожидаемый доход 0.464 с риском 0.00615

Итерация: 18
Ожидаемый доход 0.464 с риском 0.00615

Ите

[None, None, None, None, None]

In [396]:
print('Веса(%) в рекомендуемом портфеле:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(5))]

print('\nОжидаемый доход {} с риском {}\n'.format(Expected_returns,Expected_risk))

Веса(%) в рекомендуемом портфеле:

BABA : -2201.172300960458
RHM.DE : -5.845510821530581
NVDA : 4188.43849648327
AFLT.ME : -4661.072250980149
MOEX.ME : 2680.6515662788843

Ожидаемый доход 17.56164943533257 с риском 52246.99213015754

